In [81]:
import os
import sys
sys.path.insert(0, r'../../../quetzal')
from quetzal.os.parallel_call import parallel_call_notebook
import itertools
import pandas as pd
if not os.path.exists('log/'):
    os.makedirs('log/')

In [82]:
kwargs = {'workers':6, 'errout_suffix':True, 'sleep':1,'stdout_path':r'log/out.txt', 'stderr_path':r'log/err.txt'}


In [83]:
catchment_radius={'bus':'500','subway':'800','rail':'1000'}
footpaths = {'max_length':'1000', 'speed':'2.8','n_ntlegs':'2'}
params = {'catchment_radius':catchment_radius,'footpaths':footpaths}
scenarios=['montreal', 'montreal_2', 'montreal_3', 'jakarta', 'auckland', 'vancouver']

montreal: pt, road and population <br>
montreal_2: pt and population <br>
montreal_3: only pt

In [84]:
arg_list = []
for scen in scenarios:
    path = f'../../scenarios/{scen}'.format(scen=scen)
    arg = str({'training_folder': path, 'params':params}).replace("'",'"')
    arg_list.append(arg)

delete all outputs

In [85]:
for scen in scenarios:
    path = f'../../scenarios/{scen}/outputs/'.format(scen=scen)
    files = os.listdir(path)
    for file in files:
        os.remove(path+file)

In [77]:
parallel_call_notebook('main.ipynb', arg_list=arg_list, **kwargs)

[NbConvertApp] Converting notebook main.ipynb to python
[NbConvertApp] Writing 27383 bytes to main.py


0 main.py {"training_folder": "../../scenarios/base", "params": {"catchment_radius": {"bus": "500", "subway": "800", "rail": "1000"}}}
32 seconds


In [75]:
parallel_call_notebook('pathfinder.ipynb', arg_list=arg_list, **kwargs)

[NbConvertApp] Converting notebook pathfinder.ipynb to python
[NbConvertApp] Writing 7980 bytes to pathfinder.py


0 pathfinder.py {"training_folder": "../../scenarios/base", "params": {"catchment_radius": {"bus": "500", "subway": "800", "rail": "1000"}}}
42 seconds


In [78]:
expected_files = ['route_type_metrics.csv','population_mesh.geojson','route_id_metrics.csv','nodes.geojson']
for scen in scenarios:
    path = f'../../scenarios/{scen}/outputs/'.format(scen=scen)
    files = os.listdir(path)
    for file in expected_files:
        assert file in files


In [79]:
# check there is no NaN in csv
# if no population.geojson dont check catchment as it could be NaN (its ok)
for scen in scenarios:
    path = f'../../scenarios/{scen}/outputs/'.format(scen=scen)
    test = pd.read_csv(path+'route_type_metrics.csv')
    if 'population.geojson' not in os.listdir(f'../../scenarios/{scen}/inputs/'.format(scen=scen)):
        test = test.drop(columns=['catchment'])
    assert not test.isnull().any().any()


AssertionError: 

In [18]:
# check there is no NaN in csv
# if no population.geojson dont check catchment as it could be NaN (its ok)
for scen in scenarios:
    path = f'../../scenarios/{scen}/outputs/'.format(scen=scen)
    test = pd.read_csv(path+'route_id_metrics.csv')
    if 'population.geojson' not in os.listdir(f'../../scenarios/{scen}/inputs/'.format(scen=scen)):
        test = test.drop(columns=['catchment'])
    assert not test.isnull().any().any()


In [76]:
# test that pathfinder produced od_route.geojson file if od are provided.
for scen in scenarios:
    outputs = f'../../scenarios/{scen}/outputs/'.format(scen=scen)
    inputs = f'../../scenarios/{scen}/inputs/'.format(scen=scen)
    if 'od' in os.listdir(inputs):
        print(scen)
        od = gpd.read_file(inputs+'od/od.geojson')
        test = gpd.read_file(outputs+'od_route.geojson')
        assert set(test['od_name'].unique()) == set(od['name'].unique())


base
